In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
from zipfile import ZipFile

os.chdir('..')
datasets = ['CSCD', 'CSCD-Toy', 'HRSCD', 'HRSCD-Toy', 'HiUCD-Mini' , 'LEVIR-CD', 'LEVIR-CD-Toy']

for dataset in datasets:
  os.makedirs(dataset, exist_ok=True)

  source_path = f'content/drive/Othercomputers/My Laptop/RP/data/{dataset}.zip'
  destination_path = f'content/{dataset}.zip'

  shutil.copy(source_path, destination_path)

  with ZipFile(destination_path, 'r') as zip_ref:
      zip_ref.extractall(dataset)

  print(f'{dataset} copied and unpacked successfully.')

In [ ]:
os.chdir(os.path.join("content", "drive", "Othercomputers", "My Laptop", "RP", "urban-change-detection"))

In [ ]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 40.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cac

In [ ]:
!sudo apt install msttcorefonts -qq -y
!rm ~/.cache/matplotlib -rf

The following additional packages will be installed:
  cabextract distro-info libfontenc1 libmspack0 python3-chardet
  python3-debconf python3-debian python3-distro-info python3-distupgrade
  python3-update-manager python3-yaml ubuntu-advantage-tools ubuntu-pro-client
  ubuntu-pro-client-l10n ubuntu-release-upgrader-core update-manager-core
  update-notifier-common xfonts-encodings xfonts-utils zstd
Suggested packages:
  shunit2
The following NEW packages will be installed:
  cabextract distro-info libfontenc1 libmspack0 python3-chardet
  python3-debconf python3-debian python3-distro-info python3-distupgrade
  python3-update-manager python3-yaml ttf-mscorefonts-installer
  ubuntu-advantage-tools ubuntu-pro-client ubuntu-pro-client-l10n
  ubuntu-release-upgrader-core update-manager-core update-notifier-common
  xfonts-encodings xfonts-utils zstd
0 upgraded, 21 newly installed, 0 to remove and 45 not upgraded.
Need to get 2,350 kB of archives.
After this operation, 9,831 kB of additional

In [ ]:
EXPERIMENT_NAME = 'HIUCD-100-Epochs'
RELATIVE_TO_DATASET = os.path.join('..', '..', '..',  '..', '..', '..')
EPOCHS = 100
FP_MODIFIER = 1
BATCH_SIZE = 2
DATASET_NAME_DIR = 'HiUCD-Mini'
DIR = os.path.join(RELATIVE_TO_DATASET, DATASET_NAME_DIR)
DATASET_NAME = 'HIUCD'
RESTORE_PREV = True
GENERATE_PLOTS = True

NameError: name 'os' is not defined

In [ ]:
!python experiment.py --experiment_name=$EXPERIMENT_NAME --epochs=$EPOCHS --fp_modifier=$FP_MODIFIER --batch_size=$BATCH_SIZE --dir=$DIR --dataset_name=$DATASET_NAME --restore_prev=$RESTORE_PREV

Figure(1500x500)
Figure(1000x800)
Figure(2100x500)
Figure(1500x500)
Figure(1000x800)
Figure(2100x500)
Figure(1500x500)
Figure(1000x800)
Figure(2100x500)
Figure(1500x500)
Figure(1000x800)
Figure(2100x500)
Figure(800x500)
Figure(1500x1000)
/content/drive/Othercomputers/My Laptop/RP/urban-change-detection/visualization/plots.py:276: UserWarning: You passed a edgecolor/edgecolors ('w') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  plt.scatter(ground_truth, ground_truth, c='black', label='GT', edgecolors='w', s=100, marker='x')
/content/drive/Othercomputers/My Laptop/RP/urban-change-detection/visualization/plots.py:279: UserWarning: You passed a edgecolor/edgecolors ('w') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  plt.scatter(ground_truth, ground_truth, c='black', edgecolors='w', s=100, marker='x')
Figure(8

In [ ]:
EXPERIMENT_NAME = 'CSCD-100-Epochs'
RELATIVE_TO_DATASET = os.path.join('..', '..', '..',  '..', '..', '..')
EPOCHS = 100
FP_MODIFIER = 10
BATCH_SIZE = 64
DATASET_NAME_DIR = 'CSCD'
DIR = os.path.join(RELATIVE_TO_DATASET, DATASET_NAME_DIR)
DATASET_NAME = 'CSCD'
RESTORE_PREV = True
GENERATE_PLOTS = True

In [ ]:
!python experiment.py --experiment_name=$EXPERIMENT_NAME --epochs=$EPOCHS --fp_modifier=$FP_MODIFIER --batch_size=$BATCH_SIZE --dir=$DIR --dataset_name=$DATASET_NAME --restore_prev=$RESTORE_PREV --generate_plots=$GENERATE_PLOTS

Figure(1500x500)
Figure(1000x800)
Figure(2100x500)
Figure(1500x500)
Figure(1000x800)
Figure(2100x500)
Figure(1500x500)
Figure(1000x800)
Figure(2100x500)
Figure(1500x500)
Figure(1000x800)
Figure(2100x500)
Figure(800x500)
Figure(1500x1000)
/content/drive/Othercomputers/My Laptop/RP/urban-change-detection/visualization/plots.py:276: UserWarning: You passed a edgecolor/edgecolors ('w') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  plt.scatter(ground_truth, ground_truth, c='black', label='GT', edgecolors='w', s=100, marker='x')
/content/drive/Othercomputers/My Laptop/RP/urban-change-detection/visualization/plots.py:279: UserWarning: You passed a edgecolor/edgecolors ('w') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  plt.scatter(ground_truth, ground_truth, c='black', edgecolors='w', s=100, marker='x')
Figure(8

In [ ]:
EXPERIMENT_NAME = 'LEVIR-100-Epochs'
RELATIVE_TO_DATASET = os.path.join('..', '..', '..',  '..', '..', '..')
EPOCHS = 100
FP_MODIFIER = 10
BATCH_SIZE = 4
DATASET_NAME_DIR = 'LEVIR-CD'
DIR = os.path.join(RELATIVE_TO_DATASET, DATASET_NAME_DIR)
DATASET_NAME = 'LEVIR'
RESTORE_PREV = True
GENERATE_PLOTS = True

In [ ]:
!python experiment.py --experiment_name=$EXPERIMENT_NAME --epochs=$EPOCHS --fp_modifier=$FP_MODIFIER --batch_size=$BATCH_SIZE --dir=$DIR --dataset_name=$DATASET_NAME --restore_prev=True  --generate_plots=$GENERATE_PLOTS

Figure(1500x500)
Figure(1000x800)
Figure(1500x500)
Figure(1000x800)
Figure(1500x500)
Figure(1000x800)
Figure(1500x500)
Figure(1000x800)
Figure(800x500)
/content/drive/Othercomputers/My Laptop/RP/urban-change-detection/visualization/plots.py:276: UserWarning: You passed a edgecolor/edgecolors ('w') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  plt.scatter(ground_truth, ground_truth, c='black', label='GT', edgecolors='w', s=100, marker='x')
/content/drive/Othercomputers/My Laptop/RP/urban-change-detection/visualization/plots.py:279: UserWarning: You passed a edgecolor/edgecolors ('w') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  plt.scatter(ground_truth, ground_truth, c='black', edgecolors='w', s=100, marker='x')
Figure(800x500)
